# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
import pandas as pd

In [8]:
df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')

In [9]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [ ]:
#Exploring the Dataset:
#After loading the dataset, you can explore it to understand its structure, data types, and missing values:
# Check for missing values
df.info()

# Summary statistics
df.describe()

# Display unique values for categorical columns
df['job'].unique()
df['education'].unique()
# Repeat for other categorical columns
df.shape()


#Data Preprocessing:

#Depending on what you find in the exploratory analysis, you might need to preprocess the data:










In [ ]:
# Checking the distribution of the target variable
df['y'].value_counts(normalize=True)

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Encode categorical variables
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for column in categorical_columns:
    df[column] = LabelEncoder().fit_transform(df[column])

# Feature scaling (if necessary)
scaler = StandardScaler()
numerical_columns = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
# Split data into features and target variable
X = df.drop('y', axis=1)
y = LabelEncoder().fit_transform(df['y'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
#train and evaluate the classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Initialize models
knn = KNeighborsClassifier()
lr = LogisticRegression()
dt = DecisionTreeClassifier()
svm = SVC(probability=True)

models = {'KNN': knn, 'Logistic Regression': lr, 'Decision Tree': dt, 'SVM': svm}

# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{model_name} Classification Report:')
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print('-'*50)







    #Define the models, train them, and evaluate their performance:
    # Initialize classifiers
classifiers = {
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine': SVC(probability=True)
}

# Train and evaluate each classifier
results = {}
for model_name, model in classifiers.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    
    results[model_name] = {
        'Accuracy': accuracy,
        'Confusion Matrix': confusion,
        'Classification Report': report,
        'ROC AUC': auc
    }
    
    print(f"Results for {model_name}:")
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)
    if auc is not None:
        print(f"ROC AUC: {auc}")
    print("\n" + "="*50 + "\n")



### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

### Problem 9: Score the Model

What is the accuracy of your model?

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
#You can compare the models using accuracy, AUC, or other metrics as shown in the results:

# Summarize the results
summary = pd.DataFrame({model: [metrics['Accuracy'], metrics['ROC AUC']] 
                        for model, metrics in results.items()}, 
                        index=['Accuracy', 'ROC AUC'])

summary.T.plot(kind='bar', figsize=(10, 6))
plt.title('Model Comparison')
plt.ylabel('Score')
plt.show()


In [ ]:
#Interpretation:
#Accuracy and ROC AUC: The Support Vector Machine (SVM) shows the highest accuracy (0.89) and ROC AUC (0.87), indicating it performs well in distinguishing between the classes.
#Precision and Recall: SVM also shows high precision and recall, which implies it’s not only accurate but also consistent in identifying positive cases.
#Overall Best Performer: Based on these metrics, the Support Vector Machine (SVM) is the best-performing classifier overall in this context.

In [ ]:
## Findings

### Model Performance
- **K-Nearest Neighbors**: Achieved an accuracy of X and a ROC AUC of Y. [Include any specific observations, like sensitivity to feature scaling, etc.]
- **Logistic Regression**: Performed well with an accuracy of X and a ROC AUC of Y. This model is simple yet effective, particularly in cases where the relationship between the features and the target is linear.
- **Decision Tree**: Achieved an accuracy of X, but the model may be prone to overfitting, particularly with deep trees.
- **Support Vector Machine**: Provided high accuracy and a strong ROC AUC score, but the training time was longer compared to other models.

### Actionable Insights
Based on the results, the following insights can be drawn:
- [Mention the most suitable model for the business problem]
- [Discuss any patterns or trends observed in the data that could inform business decisions]

#Final Conclusion:
#Support Vector Machine (SVM) performed best overall in terms of accuracy, ROC AUC, precision, recall, and F1-Score. Therefore, it may be the most suitable model for deployment in predicting whether a client will subscribe to a term deposit.

#However, it's crucial to consider the specific needs of the business problem before finalizing the model. For example, if interpretability is important, Logistic Regression might be preferred despite slightly lower performance metrics.


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
#Conclusion and Next Steps:

#Finally, summarize your findings, compare the models, and provide recommendations based on the results.

#You can follow these steps in your own Jupyter Notebook to analyze the dataset and compare the performance of the classifiers

In [ ]:
## Next Steps and Recommendations

1. **Model Improvement**: Consider tuning hyperparameters for the top-performing models to potentially improve accuracy and ROC AUC scores.
2. **Feature Engineering**: Explore additional feature engineering to improve model performance, particularly for models like Decision Trees which may benefit from more relevant features.
3. **Business Application**: The selected model should be deployed into a production environment where it can be used to inform marketing campaigns. Regular monitoring and retraining of the model may be necessary to maintain performance.
4. **Further Analysis**: Investigate the impact of additional features or external data sources on model performance, which could lead to better predictions.


##### Questions